In [1]:
!pip install python-binance

In [33]:
from binance import Client
import pandas as pd
import numpy as np
import time
import datetime

api_key = "ncaGKCQNu9KS8kEwF3Jri37YjsOkKCfLGiKZcq87iThEU3QZIWdxOyeQLUvOCPpG"
api_secret = "tvSQ36Idx7W1QwiMx5ycduhziLQZaHUbhnHZGhqlSRaF6VrkP8QqsLY3LCidthdJ"

client = Client(api_key, api_secret)

In [28]:
def get_top_symbol():
  all_pairs = pd.DataFrame(client.get_ticker())
  relev = all_pairs[all_pairs.symbol.str.contains('USDT')]
  non_lev = relev[~((relev.symbol.str.contains('UP')) | (relev.symbol.str.contains('DOWN')))]
  top_symbol = non_lev[non_lev.priceChangePercent == non_lev.priceChangePercent.max()]
  print(non_lev.priceChangePercent.max())
  top_symbol = top_symbol.symbol.values[0]
  return top_symbol

In [29]:
get_top_symbol()

9.959


'IMXUSDT'

In [30]:
def getminutedata(symbol, interval, lookback):
  frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + ' min ago KST'))
  frame = frame.iloc[:, :6]
  frame.columns  = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  frame = frame.set_index('Time')
  frame.index = pd.to_datetime(frame.index, unit='ms')
  frame = frame.astype(float)
  time_change = datetime.timedelta(hours=9)
  frame.index = frame.index + time_change
  return frame

In [31]:
xy = getminutedata(get_top_symbol(), '1m', '120')
xy

9.847


,Open,High,Low,Close,Volume
Time,,,,,
2022-06-06 15:14:00,3.190,3.190,3.184,3.185,4165.3
2022-06-06 15:15:00,3.185,3.190,3.181,3.185,8146.4
2022-06-06 15:16:00,3.185,3.188,3.185,3.186,3332.2
2022-06-06 15:17:00,3.186,3.195,3.183,3.195,21183.0
2022-06-06 15:18:00,3.194,3.198,3.192,3.197,3841.9
...,...,...,...,...,...
2022-06-06 17:09:00,3.223,3.227,3.223,3.225,2256.6
2022-06-06 17:10:00,3.223,3.229,3.222,3.229,2569.8
2022-06-06 17:11:00,3.228,3.231,3.228,3.229,5406.0


In [32]:
#How to trade certain amount not qty

round(200/xy.Close.iloc[-1]) # 200 -> amount of dollar you want to buy; so you can buy 62 tokens of tokens

62

In [34]:
def strategy(buy_amt, SL=0.985, Target=1.02, open_position=False):
  try:
    asset = get_top_symbol()
    df = getminutedata(asset, '1m', '120')
  except:
    time.sleep(61)
    asset = get_top_symbol()
    df = getminutedata(asset, '1m', '120')
  qty = round(buy_amt/df.Close.iloc[-1])
  if ((df.Close.pct_change() + 1).cumprod()).iloc[-1] > 1:
    order = client.create_order(symbol=asset,
                                side='BUY',
                                type="MARKET",
                                quantity=qty)
    print(order)
    buyprice = float(order['fills'][0]['price'])
    open_position=True
    while open_position:
      try:
        df = getminutedata(asset, "1m", "2")
      except:
        print('something went wrong. Script continues in 1 min')
        time.sleep(61)
        df = getminutedata(asset, "1m", "2")
      print(f'current close is ' + str(df.Close[-1]))
      print(f'current Target is ' + str(buyprice * Target))
      print(f'current SL is ' + str(buyprice * SL))
      if df.Close[-1] <= buyprice * SL or df.Close[-1] >= buyprice * Target:
        order = client.create_order(symbol=asset,
                                side='SELL',
                                type="MARKET",
                                quantity=qty)
        print(order)
        break

In [ ]:
while True:
  strategy(200)

In [35]:
#USING BINANCE SOCKET MANAGER

In [36]:
def createframe(msg):
  df = pd.DataFrame([msg])
  df = df.loc[:, ['s', 'E', 'p']]
  df.columns = ['symbol', 'Time', 'Price']
  df.Price = df.Price.astype(float)
  df.Time = pd.to_datetime(df.Time, unit='ms')
  time_change = datetime.timedelta(hours=9)
  df.Time = df.Time + time_change 
  return df

In [37]:
from binance import BinanceSocketManager

In [38]:
bsm = BinanceSocketManager(client)

In [39]:
async def strategy(buy_amt, SL=0.985, Target=1.02, open_position=False):
  try:
    asset = get_top_symbol()
    df = getminutedata(asset, '1m', '120')
  except:
    time.sleep(61)
    asset = get_top_symbol()
    df = getminutedata(asset, '1m', '120')
  qty = round(buy_amt/df.Close.iloc[-1])
  if ((df.Close.pct_change() + 1).cumprod()).iloc[-1] > 1:
    order = client.create_order(symbol=asset,
                                side='BUY',
                                type="MARKET",
                                quantity=qty)
    print(order)
    buyprice = float(order['fills'][0]['price'])
    open_position=True
    while open_position:
      await socket.__aenter__()
      msg = await socket.recv()
      df = createframe(msg)
      print(f'current close is ' + str(df.Price.values))
      print(f'current Target is ' + str(buyprice * Target))
      print(f'current SL is ' + str(buyprice * SL))
      if df.Price.values <= buyprice * SL or df.Price.values >= buyprice * Target:
        order = client.create_order(symbol=asset,
                                side='SELL',  
                                type="MARKET",
                                quantity=qty)
        print(order)
        break

In [ ]:
while True:
  await strategy(200)